<a href="https://colab.research.google.com/github/Confierka/pythonProject_HSE/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading dataframes

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from plotly import subplots

In [ ]:
df = pd.read_csv ('/content/drive/MyDrive/Collab1/imdb_top_1000.csv')

In [ ]:
df_inflation = pd.read_csv('/content/drive/MyDrive/Collab1/inflation_data_1701-2022.csv')

# Data Clenup

In [ ]:
df_inflation.amount = df_inflation.iloc[-1].amount / df_inflation.amount

In [ ]:
df.drop(['Poster_Link', 'Overview','Star1','Star2','Star3','Star4'], axis=1, inplace=True)

In [ ]:
df.drop(df[df.Released_Year.str.isnumeric() == False].index, inplace=True)

In [ ]:
df.Released_Year = df.Released_Year.apply(int)

In [ ]:
df = df[df["Gross"].isna() != True]
df = df[df["Meta_score"].isna() != True]
df = df[df["Certificate"].isna() != True]

In [ ]:
def cleanRuntime (s):
  return int(s.split()[0])

df['Runtime'] = df['Runtime'].apply(cleanRuntime)

In [ ]:
def clean_genre(genre):
  return set(
      g.strip().strip(',').lower() for g in genre.split()
  )
df['Genre'] = df['Genre'].apply(clean_genre)

In [ ]:
df['Gross'] = df["Gross"].apply(str)
def cleanGross2(s):
  return s.replace(",", "")
df['Gross'] = df["Gross"].apply(cleanGross2)
df['Gross'] = df['Gross'].apply(int)

In [ ]:
df['Dollar_price'] = pd.merge (
    left = df.reset_index(),
    right=df_inflation,
    how='left',
    left_on ="Released_Year",
    right_on='year'
).set_index('index')['amount']

In [ ]:
df['Indexed_Gross'] = df['Gross']*df['Dollar_price']

In [ ]:
df.isna().sum().sum()

0

# Statistics

In [ ]:
MeanRating = df['IMDB_Rating'].mean()
MedianRating = df['IMDB_Rating'].median()
StdRating = df['IMDB_Rating'].std()

MeanRuntime = df['Runtime'].mean()
MedianRuntime = df['Runtime'].median()
StdRuntime = df['Runtime'].std()

MeanGross = df['Indexed_Gross'].mean()
MedianGross = df['Indexed_Gross'].median()
StdGross = df['Indexed_Gross'].std()


In [ ]:
print('Rating:','Mean =',MeanRating,'Median =',MedianRating,'Std =',StdRating)
print('Runtime:','Mean =',MeanRuntime,'Median =',MedianRuntime,'Std =',StdRuntime)
print('Indexed_Gross:','Mean =',MeanGross,'Median =',MedianGross,'Std =',StdGross)

Rating: Mean = 7.937587657784011 Median = 7.9 Std = 0.29321122781557785
Runtime: Mean = 123.69284712482468 Median = 120.0 Std = 25.89850906516098
Indexed_Gross: Mean = 161824289.5830974 Median = 79005959.25056434 Std = 265738666.37202737


In [ ]:
df['IMDB_Rating']  = df["IMDB_Rating"]*10.0

In [ ]:
df['OverallRating'] = (df['IMDB_Rating'] +df['Meta_score'])/2



OverallRating is mean from IMDB_rating and Meta_score

In [ ]:
df.sort_values(by='Indexed_Gross', ascending=False)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,Director,No_of_Votes,Gross,Dollar_price,Indexed_Gross,OverallRating
314,Gone with the Wind,1939,U,238,"{history, drama, romance}",81.0,97.0,Victor Fleming,290074,198676459,20.074303,3.988292e+09,89.0
875,Fantasia,1940,G,125,"{animation, fantasy, family}",77.0,96.0,James Algar,88662,76408097,19.889571,1.519724e+09,86.5
29,Star Wars,1977,UA,121,"{action, fantasy, adventure}",86.0,90.0,George Lucas,1231473,322740140,4.601845,1.485200e+09,88.0
419,The Exorcist,1973,A,122,{horror},80.0,81.0,William Friedkin,362393,232906145,6.276864,1.461920e+09,80.5
429,The Sound of Music,1965,U,172,"{drama, family, biography}",80.0,63.0,Robert Wise,205425,163214286,8.845839,1.443767e+09,71.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,The Boondock Saints,1999,R,108,"{action, crime, thriller}",78.0,44.0,Troy Duffy,227143,25812,1.673722,4.320212e+04,61.0
239,Salinui chueok,2003,UA,131,"{crime, drama, mystery}",81.0,82.0,Bong Joon Ho,139558,14131,1.515307,2.141281e+04,81.5
356,Tropa de Elite,2007,R,115,"{action, crime, drama}",80.0,33.0,José Padilha,98097,8060,1.344670,1.083804e+04,56.5
624,Mr. Nobody,2009,R,141,"{drama, romance, fantasy}",78.0,63.0,Jaco Van Dormael,216421,3600,1.299659,4.678773e+03,70.5


# Simple plots

In [ ]:
fig = px.scatter(df, x='IMDB_Rating',y="Meta_score",range_x=(1,100),range_y=(1,100),color='Indexed_Gross',trendline='ols',title = 'Movies Rating corelation')
fig2 = px.line(x=[1,10],y=[1,100])
fig.show()

In [ ]:
year_cnt = int(df['Released_Year'].max() - df['Released_Year'].min()+1)
fig1 = px.histogram(df, 'Released_Year',nbins=year_cnt, title = 'Count movies per year')
fig1.show()

In [ ]:
fig = px.pie(df, names = 'Certificate', title = 'Movies by their certificate')
fig.show()

# Detailed plots

In [ ]:
full_genre_list = (set.union(*list(df.Genre)))
print(*full_genre_list, sep='\n')

adventure
crime
film-noir
thriller
western
fantasy
animation
comedy
romance
sci-fi
drama
mystery
musical
sport
war
family
history
horror
action
music
biography


In [ ]:
def get_genre_subset(df,genre):
  return df[df.Genre.apply(lambda x: genre in x)]

genres = ['drama', 'action','crime','horror']

In [ ]:
fig = subplots.make_subplots(rows=len(genres),cols=1, subplot_titles=genres)

for i,g in enumerate(genres):
  fig_s = px.histogram(get_genre_subset(df, g), 'Released_Year', nbins=year_cnt)
  for d in fig_s['data']:
    fig.add_trace(d, row = i + 1, col = 1)
  fig.update_yaxes(range=[0,20],row=i+1,col=1)
  fig.update_xaxes(range=[df.Released_Year.min(),df.Released_Year.max()])


fig.update_layout(title_text = 'Count of movies in genre')

fig.show()

In [ ]:
fig = subplots.make_subplots(rows=len(genres),cols=1, subplot_titles=genres)

for i,g in enumerate(genres):
  fig_s = px.histogram(get_genre_subset(df, g), 'Released_Year', y="Indexed_Gross", nbins=year_cnt)
  for d in fig_s['data']:
    fig.add_trace(d, row = i + 1, col = 1)
  fig.update_yaxes(range=[0,5_000_000_000],row=i+1,col=1)
  fig.update_xaxes(range=[df.Released_Year.min(),df.Released_Year.max()])


fig.update_layout(title_text = 'Gross of movies in genre')

fig.show()

#Hypothesis check



Hypotheisis: Gross is correlating with IMDB_Rating and Meta_Score

In [ ]:
rating_gross_corr = df[['Indexed_Gross', 'IMDB_Rating','Meta_score']].corr()


In [ ]:
imdb_gross_coer = rating_gross_corr.IMDB_Rating['Indexed_Gross']
Metascore_gross_coer = rating_gross_corr.Meta_score['Indexed_Gross']

In [ ]:
fig = subplots.make_subplots(rows=1,cols=2, subplot_titles=[f'coerr={imdb_gross_coer:.3f}',f'coerr={Metascore_gross_coer:.3f}'])

fig1 = px.scatter(df, x='IMDB_Rating',y="Gross",trendline='ols', color = 'Meta_score')
fig2 = px.scatter(df, x='Meta_score',y="Gross",trendline='ols',color = 'IMDB_Rating')

fig1.update_traces(line_color='black')
fig2.update_traces(line_color='black')


for d in fig1['data']:
  fig.add_trace(d, row = 1, col = 1)
for d in fig2['data']:
  fig.add_trace(d, row = 1, col = 2)

fig.update_xaxes(range=[1,100],row=1,col=1)
fig.update_xaxes(range=[1,100],row=1,col=2)

fig.update_layout(title_text = 'IMDB and Meta Ration to gross')

fig.show()

Hypthesis has not been confirmed, there are very little correlations between rating and gross